# 1. Configuraciones

In [1]:
# Características de la MV
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
# Declarar variables
HOME ="/content/drive/MyDrive/Maestria/Tesis/DFL-BundesligaDataShootout/Experimentacion/test"
HOME

'/content/drive/MyDrive/Maestria/Tesis/DFL-BundesligaDataShootout/Experimentacion/test'

# 2. Conectarse a drive

In [3]:
# Conectarse a drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Establecer directorio de trabajo
%cd {HOME}

/content/drive/MyDrive/Maestria/Tesis/DFL-BundesligaDataShootout/Experimentacion/test


In [9]:
!ls -l

total 107
-rw------- 1 root root 69177 Mar 12 15:11 '3. Image Annotation.ipynb'
-rw------- 1 root root 39441 Mar 12 14:22  annotation_video_1606b0e6_0.xlsx


# 3. Extraer imágenes de video

In [10]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.2 MB/s eta 0:00:00


In [11]:
import glob
import os
import numpy as np
import math
import pandas as pd
import torch
import cv2

from tqdm import tqdm

torch.backends.cudnn.benchmark = True

from timm.data import ImageDataset, create_loader

In [ ]:
## 3.1 Extracción de una imagen x segundo

In [ ]:
vclips = sorted(glob.glob('../data/clips/*'))
vclips[0:5]

['./data/clips/08fd33_0.mp4',
 './data/clips/08fd33_1.mp4',
 './data/clips/08fd33_2.mp4',
 './data/clips/08fd33_3.mp4',
 './data/clips/08fd33_4.mp4']

In [ ]:
vclips[0]

'./data/clips/08fd33_0.mp4'

In [ ]:
def extract_images(video_path, out_dir, img_size):
    """
    Función para extraer imágenes de un video, se extrae una imagen x segundo.
    video_path = Dirección del video del cual se quiere extraer los frames o imágenes.
    out_dir = directorio dónde se almacenarán la ímagenes extraídas del video.
    img_size = tamaño de las imágenes de salida.
    """
    video_name = os.path.basename(video_path).split('.')[0] # Extraemos la ruta dónde esta el video.
    cam = cv2.VideoCapture(video_path) # Preparamos para la captura
    fps = int(cam.get(cv2.CAP_PROP_FPS)) # Cantidad de FPS
    print(video_path, fps)
    frame_count = 0  # Contador
    while True:
        current_fps = int(cam.get(1)) # Frame actual
        successed, img = cam.read() # Leemos el frame actual
        # Verificamos que el frame del cual se va a generar la imagen, corresponde al segundo siguiente.
        if (current_fps % math.floor(fps)==0):
            outfile = f'{out_dir}/{video_name}-second{int(current_fps/fps)}.jpg' # Archivos de salida
            img = cv2.resize(img, dsize=img_size, interpolation=cv2.INTER_AREA) # Redimensianamos la imagen
            cv2.imwrite(outfile, img) # Guardamos la imagen
        if not successed:
            break # Si ya no hay más frames paramos
        frame_count += 1

In [ ]:
%%time
video_path = vclips[0]
out_dir = '../images'
img_size = (544,544)

extract_images(video_path, out_dir, img_size)

## 3.2 Prueba extracción imágenes de un video de train.

In [19]:
# Función para leer y editar la data de las anotaciones en train.
def image_annotation(path_data):
    """
    Función para leer y editar la data de las anotaciones de jugadas en un video dado.
    path_data = Es la ruta del directorio dónde se encuentra el archivo csv.
    """
    # Leemos el archivo train.csv con las anotaciones.
    train = pd.read_csv(path_data)
    # Redondeamos time al segundo más cercano
    train['second'] = [int(np.round(x,0)) for x in train.time]
    # Construimos el id de la imagen
    train['image_id'] = train.video_id + '-second' + train.second.astype(str)
    # Construímos el label o etiqueta de cada imagen
    train['label0'] = np.where(train.event_attributes.isna(), np.nan, train.event) # Label = event is event != start or end else nan
    train['label1'] = train['label0'].fillna(method = 'backfill') # Reemplazamos na de labels por el valor anterior
    train['label2'] = train['label0'].fillna(method = 'ffill') # Reemplazamos na de labels por el valor posterior
    # Implementamos lógica de imputación 
    train['label'] = np.where(train.event == 'start', train.label1,
                            np.where(train.event=='end',train.label2, train.label0))
    # Eliminamos imagenes duplicadas.
    train = train.drop_duplicates(['image_id'], keep = 'first')
    # Seleccionamos columnas necesarias
    dict_labels = dict(zip(train.image_id, train.label))
    train = train.drop(['label0', 'label1', 'label2'], axis = 1)#[['image_id', 'label']].copy()
    return train, dict_labels

In [20]:
path_data = '../data/train.csv'
train, dict_labels = image_annotation(path_data)
print(train.shape)
train.head(20)

(9870, 7)


,video_id,time,event,event_attributes,second,image_id,label
0,1606b0e6_0,200.265822,start,NaN,200,1606b0e6_0-second200,challenge
1,1606b0e6_0,201.150000,challenge,['ball_action_forced'],201,1606b0e6_0-second201,challenge
2,1606b0e6_0,202.765822,end,NaN,203,1606b0e6_0-second203,challenge
3,1606b0e6_0,210.124111,start,NaN,210,1606b0e6_0-second210,challenge
4,1606b0e6_0,210.870000,challenge,['opponent_dispossessed'],211,1606b0e6_0-second211,challenge
5,1606b0e6_0,212.624111,end,NaN,213,1606b0e6_0-second213,challenge
6,1606b0e6_0,217.850213,start,NaN,218,1606b0e6_0-second218,throwin
7,1606b0e6_0,219.230000,throwin,['pass'],219,1606b0e6_0-second219,throwin
8,1606b0e6_0,220.350213,end,NaN,220,1606b0e6_0-second220,throwin
9,1606b0e6_0,223.930850,start,NaN,224,1606b0e6_0-second224,play


In [21]:
dict_labels

{'1606b0e6_0-second200': 'challenge',
 '1606b0e6_0-second201': 'challenge',
 '1606b0e6_0-second203': 'challenge',
 '1606b0e6_0-second210': 'challenge',
 '1606b0e6_0-second211': 'challenge',
 '1606b0e6_0-second213': 'challenge',
 '1606b0e6_0-second218': 'throwin',
 '1606b0e6_0-second219': 'throwin',
 '1606b0e6_0-second220': 'throwin',
 '1606b0e6_0-second224': 'play',
 '1606b0e6_0-second226': 'play',
 '1606b0e6_0-second229': 'play',
 '1606b0e6_0-second231': 'play',
 '1606b0e6_0-second236': 'play',
 '1606b0e6_0-second237': 'play',
 '1606b0e6_0-second239': 'play',
 '1606b0e6_0-second240': 'play',
 '1606b0e6_0-second242': 'play',
 '1606b0e6_0-second245': 'play',
 '1606b0e6_0-second246': 'play',
 '1606b0e6_0-second247': 'play',
 '1606b0e6_0-second248': 'play',
 '1606b0e6_0-second249': 'play',
 '1606b0e6_0-second250': 'play',
 '1606b0e6_0-second251': 'play',
 '1606b0e6_0-second253': 'play',
 '1606b0e6_0-second254': 'play',
 '1606b0e6_0-second255': 'play',
 '1606b0e6_0-second257': 'play',
 '16

In [22]:
    train.label.value_counts()

play         8004
challenge    1431
throwin       435
Name: label, dtype: int64

In [23]:
!mkdir -p images/background images/challenge images/play images/throwin

In [24]:
def extract_images(video_path, out_dir, img_size, dict_labels):
    """
    Función para extraer imágenes de un video, se extrae una imagen x segundo.
    video_path = Dirección del video del cual se quiere extraer los frames o imágenes.
    out_dir = directorio dónde se almacenarán la ímagenes extraídas del video.
    img_size = tamaño de las imágenes de salida.
    """
    video_name = os.path.basename(video_path).split('.')[0] # Extraemos la ruta dónde esta el video.
    cam = cv2.VideoCapture(video_path) # Preparamos para la captura
    fps = int(cam.get(cv2.CAP_PROP_FPS)) # Cantidad de FPS
    print(video_path, fps)
    frame_count = 0  # Contador
    while True:
        current_fps = int(cam.get(1)) # Frame actual
        successed, img = cam.read() # Leemos el frame actual
        # Verificamos que el frame del cual se va a generar la imagen, corresponde al segundo siguiente.
        if (current_fps % math.floor(fps)==0):
            image_id = f'{video_name}-second{int(current_fps/fps)}'
            if image_id in dict_labels.keys():
                outfile = f'{out_dir}/{dict_labels[image_id]}/{image_id}.jpg' # Archivos de salida
            else:
                outfile = f'{out_dir}/background/{image_id}.jpg' # Archivos de salida
            
            img = cv2.resize(img, dsize=img_size, interpolation=cv2.INTER_AREA) # Redimensianamos la imagen
            cv2.imwrite(outfile, img) # Guardamos la imagen
        if not successed:
            break # Si ya no hay más frames paramos
        frame_count += 1

In [25]:
train_videos = sorted(glob.glob('../data/train/*'))
train_videos#[0:5]

['../data/train/1606b0e6_0.mp4',
 '../data/train/1606b0e6_1.mp4',
 '../data/train/35bd9041_0.mp4',
 '../data/train/35bd9041_1.mp4',
 '../data/train/3c993bd2_0.mp4',
 '../data/train/3c993bd2_1.mp4',
 '../data/train/407c5a9e_1.mp4',
 '../data/train/4ffd5986_0.mp4',
 '../data/train/9a97dae4_1.mp4',
 '../data/train/cfbe2e94_0.mp4',
 '../data/train/cfbe2e94_1.mp4',
 '../data/train/ecf251d4_0.mp4']

In [26]:
train_videos[0]

'../data/train/1606b0e6_0.mp4'

In [27]:
%%time
video_path = train_videos[0]
out_dir = './images'
img_size = (544,544)

extract_images(video_path, out_dir, img_size, dict_labels)

../data/train/1606b0e6_0.mp4 25
CPU times: user 21min 15s, sys: 33.8 s, total: 21min 48s
Wall time: 15min 43s


In [28]:
# Data de las anotaciones
train[train.video_id=='1606b0e6_0'].to_excel(f'{HOME}/annotation_video_1606b0e6_0.xlsx', index = False)